In [148]:
from pathlib import Path
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString, MultiLineString, Point
from shapely.ops import substring
import folium
import branca.colormap as cm


OUT = Path("outputs")
GPKG_CROWNS = OUT / "crowns.gpkg"                 # from nb02
GPKG_LINES  = OUT / "highways_aoi.gpkg"         # from nb01

assert GPKG_LINES.exists(), "Missing outputs/highways_aoi.gpkg (run Notebook 01)"
assert GPKG_CROWNS.exists(), "Missing outputs/crowns.gpkg (run Notebook 02)"

print("OK:", GPKG_LINES, GPKG_CROWNS)


OK: outputs/highways_aoi.gpkg outputs/crowns.gpkg


In [149]:
lines = gpd.read_file(GPKG_LINES, layer="lines")
lines_buf = gpd.read_file(GPKG_LINES, layer="lines_buffer")
crowns_all = gpd.read_file(GPKG_CROWNS, layer="crowns_all")
crowns_near = gpd.read_file(GPKG_CROWNS, layer="crowns_near_lines")

# unify CRS
crs = lines.crs
lines = lines.to_crs(crs)
lines_buf = lines_buf.to_crs(crs)
crowns_all = crowns_all.to_crs(crs)
crowns_near = crowns_near.to_crs(crs)

print("CRS:", crs)
print("lines:", len(lines), "buffers:", len(lines_buf), "crowns:", len(crowns_all), "near:", len(crowns_near))


CRS: EPSG:26910
lines: 59 buffers: 59 crowns: 3455 near: 0


In [150]:
def split_line_to_segments(line: LineString, seg_len: float):
    """Split a LineString into segments of length ~seg_len (last one may be shorter)."""
    if line.length <= 0:
        return []
    segs = []
    dist = 0.0
    while dist < line.length:
        start = dist
        end = min(dist + seg_len, line.length)
        seg = substring(line, start, end, normalized=False)
        # substring can return Point for zero-length; skip those
        if isinstance(seg, LineString) and seg.length > 0:
            segs.append(seg)
        dist = end
    return segs

def explode_to_segments(gdf_lines: gpd.GeoDataFrame, seg_len_m: float) -> gpd.GeoDataFrame:
    rows = []
    for idx, row in gdf_lines.iterrows():
        geom = row.geometry
        if geom is None:
            continue
        geoms = []
        if isinstance(geom, LineString):
            geoms = split_line_to_segments(geom, seg_len_m)
        elif isinstance(geom, MultiLineString):
            for part in geom.geoms:
                geoms.extend(split_line_to_segments(part, seg_len_m))
        for i, seg in enumerate(geoms):
            r = {k: row.get(k) for k in gdf_lines.columns if k != "geometry"}
            r["span_id"] = f"{idx}_{i}"
            r["geometry"] = seg
            r["span_length_m"] = seg.length
            rows.append(r)
    return gpd.GeoDataFrame(rows, crs=gdf_lines.crs)


In [151]:
import geopandas as gpd

SEG_LEN_M = 100.0
ASSIGN_USING = "centroid"   # "centroid" or "intersect"
SPAN_BUF_M = 100.0           # max snap distance (meters)

# 0) Make sure spans have a unique id
spans = explode_to_segments(lines, SEG_LEN_M).reset_index(drop=True)
if "span_id" not in spans.columns:
    spans["span_id"] = spans.index.astype(int)

# 1) Work in meters
if not spans.crs:
    raise ValueError("spans has no CRS")
utm = spans.estimate_utm_crs() if not spans.crs.is_projected else spans.crs
spans_m  = spans.to_crs(utm)
crowns_m = crowns_near.to_crs(utm)

print("spans:", len(spans_m), "| total length (km):", spans_m.length.sum()/1000.0)

if ASSIGN_USING == "centroid":
    # points = crown centroids
    pts = crowns_m.copy()
    pts["geometry"] = pts.geometry.centroid

    # nearest join within SPAN_BUF_M (meters)
    assigned = gpd.sjoin_nearest(
        pts[["geometry"]],
        spans_m[["span_id", "geometry"]],
        how="left",
        distance_col="dist_m",
        max_distance=SPAN_BUF_M,
    )

    # assigned has same length as pts (no duplication), so this is safe:
    crowns_m["span_id"] = assigned["span_id"].to_numpy()

elif ASSIGN_USING == "intersect":
    # This may produce duplicates (one crown ↔ multiple spans).
    tmp = gpd.sjoin(
        crowns_m, spans_m[["span_id", "geometry"]],
        how="left", predicate="intersects"
    ).reset_index(names="crown_idx")

    # Pick the best match per crown: the longest overlap with the span line.
    # (intersection of polygon


spans: 161 | total length (km): 12.7121203498092


In [152]:
# counts per span
agg = (crowns_m
       .groupby("span_id")
       .size()
       .reset_index(name="crowns_near"))

# join with span length
spans_ranked = spans.merge(agg, on="span_id", how="left").fillna({"crowns_near": 0})
spans_ranked["crowns_per_100m"] = spans_ranked["crowns_near"] / (spans_ranked["span_length_m"].clip(lower=1.0)/100.0)

# simple risk score (you can tune this)
spans_ranked["risk_score"] = spans_ranked["crowns_per_100m"]

# rank
spans_ranked = spans_ranked.sort_values(["risk_score","crowns_near"], ascending=[False, False]).reset_index(drop=True)
spans_ranked.head(10)


,highway,name,osm_id,maxspeed,alt_name,bicycle,cycleway,lanes,sidewalk:both,surface,...,turn:lanes,lit,hazard,lanes:bus:backward,span_id,geometry,span_length_m,crowns_near,crowns_per_100m,risk_score
0,tertiary,99th Place Northeast,6350131,None,None,None,None,None,None,None,...,None,None,None,None,0_0,"LINESTRING (559420.545 5283485.009, 559419.22 ...",100.000000,0.0,0.0,0.0
1,tertiary,99th Place Northeast,6350131,None,None,None,None,None,None,None,...,None,None,None,None,0_1,"LINESTRING (559392.509 5283579.024, 559390.037...",100.000000,0.0,0.0,0.0
2,tertiary,99th Place Northeast,6350131,None,None,None,None,None,None,None,...,None,None,None,None,0_2,"LINESTRING (559340.205 5283664.133, 559329.253...",52.143341,0.0,0.0,0.0
3,tertiary,93rd Avenue Northeast,6357246,25 mph,None,None,None,None,None,None,...,None,None,None,None,1_0,"LINESTRING (558664.583 5283852.984, 558664.197...",100.000000,0.0,0.0,0.0
4,tertiary,93rd Avenue Northeast,6357246,25 mph,None,None,None,None,None,None,...,None,None,None,None,1_1,"LINESTRING (558664.857 5283952.972, 558664.924...",100.000000,0.0,0.0,0.0
5,tertiary,93rd Avenue Northeast,6357246,25 mph,None,None,None,None,None,None,...,None,None,None,None,1_2,"LINESTRING (558666.097 5284052.959, 558666.668...",100.000000,0.0,0.0,0.0
6,tertiary,93rd Avenue Northeast,6357246,25 mph,None,None,None,None,None,None,...,None,None,None,None,1_3,"LINESTRING (558666.941 5284152.948, 558666.728...",100.000000,0.0,0.0,0.0
7,tertiary,93rd Avenue Northeast,6357246,25 mph,None,None,None,None,None,None,...,None,None,None,None,1_4,"LINESTRING (558666.768 5284252.946, 558667.361...",100.000000,0.0,0.0,0.0
8,tertiary,93rd Avenue Northeast,6357246,25 mph,None,None,None,None,None,None,...,None,None,None,None,1_5,"LINESTRING (558667.537 5284352.943, 558667.597...",100.000000,0.0,0.0,0.0
9,tertiary,93rd Avenue Northeast,6357246,25 mph,None,None,None,None,None,None,...,None,None,None,None,1_6,"LINESTRING (558667.103 5284452.935, 558667.053...",67.029396,0.0,0.0,0.0


In [153]:
gpkg_out = OUT / "ranked_spans.gpkg"
csv_out  = OUT / "ranked_spans.csv"

# save geospatial and tabular
spans_ranked.to_file(gpkg_out, layer="spans_ranked", driver="GPKG")
spans_ranked[["span_id","span_length_m","crowns_near","crowns_per_100m","risk_score"]].to_csv(csv_out, index=False)

print("Wrote:", gpkg_out)
print("Wrote:", csv_out)


Wrote: outputs/ranked_spans.gpkg
Wrote: outputs/ranked_spans.csv


In [154]:

# project to WGS84 for web map
spans_wgs = spans_ranked.to_crs(4326)

# color scale
vmin, vmax = 0, max(1.0, spans_wgs["risk_score"].quantile(0.95))
cmap = cm.linear.YlOrRd_09.scale(vmin, vmax)

# center on spans bbox
bounds = spans_wgs.total_bounds
center = [(bounds[1]+bounds[3])/2, (bounds[0]+bounds[2])/2]

m = folium.Map(location=center, zoom_start=14, control_scale=True)
for _, r in spans_wgs.iterrows():
    val = float(r["risk_score"])
    color = cmap(val)
    folium.GeoJson(
        r.geometry.__geo_interface__,
        style_function=lambda _r, color=color: {"color": color, "weight": 4, "opacity": 0.9}
    ).add_to(m)

cmap.caption = "Risk score (crowns per 100 m)"
cmap.add_to(m)

html_map = OUT / "ranked_spans.html"
m.save(html_map)
html_map


PosixPath('outputs/ranked_spans.html')

In [155]:
summary = {
    "n_lines": int(len(lines)),
    "n_spans": int(len(spans_ranked)),
    "n_crowns": int(len(crowns_all)),
    "n_crowns_near": int(len(crowns_m)),
    "top5_avg_risk": float(spans_ranked["risk_score"].head(5).mean() if len(spans_ranked) else 0.0),
}
summary


{'n_lines': 59,
 'n_spans': 161,
 'n_crowns': 3455,
 'n_crowns_near': 0,
 'top5_avg_risk': 0.0}